

Задание:

    объединить в одну выборку
    на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов (на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)
    Проверить насколько хорошо работают подходы



In [1]:
#Загрузка данных
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-06-19 10:40:23--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-19 10:40:23--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf788216df7723680704e016ab2.dl.dropboxusercontent.com/cd/0/inline/BndDsGIisiIdipVQHkhUHJ7fyss_q6mYm0bfAoy602rRJMXLeH9tR9gCp_SPuHHwMD6tXSIOvWuraRk1TUq2kjfzQ_I2msKAmfmmXV-ARkUv-iEHifIXgg3vRfxelASpC414fBLx_-Uy-tjLs4Tq13pkrqP-uc4a1fNrW1M8cLuNZw/file# [following]
--2022-06-19 10:40:24--  https://ucf788216df7723680704e016ab2.dl.dropboxusercontent.com/cd/0/inline/BndDsGIisiIdipVQHkhUHJ7fyss_q6mYm0bfAoy602rRJMXLeH9tR9gCp_SPuHHwMD6t

In [2]:
# Установка библиотек
!pip install pymorphy2
!pip install stop_words
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Загрузка библиотек
import pandas as pd
import gensim.downloader as api
from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
from sklearn.model_selection import train_test_split
import annoy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import gensim.models
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Embedding
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tqdm import tqdm

In [4]:
#Импорт данных и объединение в один датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = 'positive'

negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = 'negative'

df = positive.append(negative)
df.sample(10)

,text,label
74088,"Четось я смотрю как мин половина тех, кто пост...",positive
94150,"Пиздато, когда у тебя новый номер и его знают ...",positive
97161,Почему? Ну почему мне всегда приходится выбира...,negative
70963,25 самых опасных для психики фильмов\nОграниче...,negative
17348,@mosssberry вотвот( теперь учусь на ошибках и ...,negative
35434,"@dasha_nechaeVa даш, ну ты че??!?!?!?! это сли...",positive
1116,Красивая)) — Воооу спасибочки:3 http://t.co/a6...,positive
22641,@hotselenurr одно вытекает из другово. соответ...,negative
36105,"час потратила, но наткнулась на то, что хотела...",positive
101621,"Хочу что бы были снегопады большие.Эйй,зимааа,...",negative


In [5]:
# Разделение на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(df.text, df.label)

In [6]:
#Предобработка данных

import nltk
nltk.download('stopwords')

import nltk
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
morpher = MorphAnalyzer()
sw = get_stop_words("ru") + stopwords.words('russian')
punkt = list(string.punctuation)
noise = set(sw + punkt)

def preprocess_txt(text):
    return [t for t in [morpher.parse(token)[0].normal_form 
                                for token in word_tokenize(text)] if t not in noise]

In [8]:
%%time
assert True

# Preprocess for models fitting
tweet_tokens = []
c = 0
for text in tqdm(X_train):
    tweet_prep = preprocess_txt(text)
    tweet_tokens.append(tweet_prep)
    c += 1
    
    if c > 100000:
        break

 59%|█████▉    | 100000/170125 [04:42<03:18, 353.47it/s]

CPU times: user 4min 32s, sys: 2.34 s, total: 4min 34s
Wall time: 4min 42s


In [9]:
# убираем из токенов X_train'a те, короче 2 символов
tweet_tokens = [i for i in tweet_tokens if len(i) > 2]

In [10]:
# Функция поиска 5 ближайших твитов
def get_response(tweet, index, model, index_map):
    tweet_token = preprocess_txt(tweet)
    vector = np.zeros(300)
    norm = 0
    for word in tweet_token:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    similar_tweet = index.get_nns_by_vector(vector, 5, include_distances=True)
    
    result = pd.DataFrame({'tweet': [index_map[i]  for i in similar_tweet[0]], 
                           'distance' : similar_tweet[1]})
    return result

In [11]:
# Процедура поиска ближайших твитов
def get_model_index(model):
    model_index = annoy.AnnoyIndex(300 ,'angular') # поиск ближайших соседей

    index_map = {}
    counter = 0

    for tweet in X_train:
        n_model = 0
        index_map[counter] = tweet
        tweet_prep = preprocess_txt(tweet)
        
        vector_model = np.zeros(300) #вектор твита

        for word in tweet_prep:
            if word in model:
                vector_model += model[word]
                n_model += 1
        if n_model > 0:
            vector_model = vector_model / n_model #нормализуем вектор твита
       
        model_index.add_item(counter, vector_model) #добавляем элемент для индексации
     
        counter += 1

        if counter > 100000:
            break

    model_index.build(10) #из индексов создаем 10 кластеров
    return model_index, index_map

**Word2Vec**

In [12]:
modelW2V = Word2Vec(sentences=tweet_tokens, size = 300, window=5, min_count=1)
w2v_index, index_map = get_model_index(modelW2V.wv)

In [13]:
print(X_test.iloc[0])
get_response(X_test.values[0], w2v_index, modelW2V.wv, index_map)

Выходной на Ура!!!:)
И все вроде супер но мало....!!!хочу еще.


,tweet,distance
0,Эх...Как все таки быстро прошли выходные:((,0.028363
1,Ну что ж....сегодняшний день прошёл довольно-т...,0.034504
2,"Вроде как и выходной сегодня,но дел куча.пришл...",0.036346
3,@_oolya Воу! Крутенько... если бы я так просп...,0.036352
4,Проспала на работку((( Опоздала....плохой день:((,0.037033


**Fasttext**

In [14]:
modelFT = FastText(sentences=tweet_tokens, size=300, min_count=1, window=5, workers=8)
ft_index, index_map = get_model_index(modelFT.wv)

In [15]:
print(X_test.iloc[0])
get_response(X_test.values[0], ft_index, modelFT.wv, index_map)

Выходной на Ура!!!:)
И все вроде супер но мало....!!!хочу еще.


,tweet,distance
0,@Artem_Nebozdes привет. Вчера был выходной!!!)...,0.052555
1,"Мдаа...\nОтличное начало дня,\n5 пар просто ши...",0.052869
2,Велосипед это здорово! Пару часиков вечерком.....,0.058243
3,Велосипед это здорово! Пару часиков вечерком.....,0.058243
4,Плюсы и минусы понедельника: \n +:\n -Танцы\n-...,0.065570


**Embedding слой**

In [16]:
text_dataset = tf.data.Dataset.from_tensor_slices([x for l in tweet_tokens for x in l]) #список всех токенов из x_train

In [17]:
# Создание слоя словаря

vectorize_layer = TextVectorization(
    max_tokens=100000,
    output_mode='int',
    output_sequence_length=1)
#создаем словарь
vectorize_layer.adapt(text_dataset)

In [18]:
#создаем модель
model = Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string)) #на вход подается строка
model.add(vectorize_layer)  #получаем тензор (1, 100), содержащий для каждого слова индексы из словаря 
model.add(Embedding(100000, 300, input_length=100, mask_zero=True))

In [19]:
model.predict([X_test.values[0]]).squeeze()

array([-1.87604427e-02, -7.56000355e-03, -8.64162296e-03, -8.82103294e-03,
       -4.90226597e-03,  1.65427960e-02,  4.59314510e-03,  1.73037089e-02,
       -1.87179223e-02, -2.33843084e-02, -2.67689582e-02,  1.83700062e-02,
       -5.20361587e-03, -1.25202909e-02,  1.08218081e-02, -2.51884591e-02,
        2.69699432e-02, -1.35995448e-04,  4.20327820e-02,  4.88635637e-02,
       -3.68692875e-02, -2.87249219e-02, -3.09357885e-02, -1.16921961e-04,
        8.71505588e-03, -3.49123962e-02, -1.94815155e-02,  1.90007351e-02,
        9.82488319e-03, -8.04698467e-03, -5.27235121e-03,  6.29782677e-03,
       -4.02356163e-02, -3.79657634e-02,  2.37519853e-02,  6.77867979e-03,
        4.64197882e-02,  4.96873371e-02,  4.97776307e-02,  2.30693482e-02,
        4.18761708e-02,  4.41130884e-02, -9.64593887e-03,  1.56892464e-03,
       -4.10956517e-02,  2.27440633e-02,  4.65472676e-02,  1.28646381e-02,
        2.25502588e-02,  1.74317025e-02, -4.07623537e-02, -2.68353950e-02,
       -2.21077092e-02, -

In [20]:
# Процедура поиска ближайших твитов
emb_index = annoy.AnnoyIndex(300 ,'angular') # поиск ближайших соседей

emb_map = {}
counter = 0

for tweet in tqdm(X_train):
    emb_map[counter] = tweet
    vector_emb = model.predict([tweet]).squeeze()
    emb_index.add_item(counter, vector_emb) #добавляем элемент для индексации
    counter += 1
        
    if counter > 100000:
        break

emb_index.build(10) #из 10 кластеров

 59%|█████▉    | 100000/170125 [1:21:03<56:50, 20.56it/s]


True

In [21]:
def get_response_emb(model, emb_index, tweet, n_similar_tweets):
    vector = model.predict([tweet])[0][0]

    similar_tweet = emb_index.get_nns_by_vector(vector, n_similar_tweets, include_distances=True)
    
    result = pd.DataFrame({'tweet': [emb_map[i]  for i in similar_tweet[0]], 
                           'distance' : similar_tweet[1]
                          })
    return result

In [22]:
print(X_test.values[0])
get_response_emb(model, emb_index, X_test.values[0], 5)

Выходной на Ура!!!:)
И все вроде супер но мало....!!!хочу еще.


,tweet,distance
0,"@lieisnotgood ты думал, что я та девочка с ава...",0.0
1,"#Смех на всю квартиру... ""Открыл"" дверь комнат...",0.0
2,Серъезная заявка на участие в соревнованиях!) ...,0.0
3,"Очень плохо, что нету инстаграма для iPad ;(",0.0
4,Ужасный новый год с температурой и под одеялом...,0.0
